# Project - CIS 662 - Text Summarization

## 1. Environment Setup:



> ### 1.1 Installation and configuration:



In [1]:
!nvidia-smi

Tue Dec 12 15:53:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers[sentencepiece]
!pip install datasets
!pip install py7zr
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-a

In [4]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.6 MB/s eta 0:00:00
Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
Found existing installation: accelerate 0.25.0
Uninstalling accelerate-0.25.0:
  Successfully uninstalled accelerate-0.25.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 24.0 MB/s eta 0:00:00
  Using cached accelerate-0.25.0-py3-none-any.whl (265 kB)


In [38]:
from transformers import pipeline, set_seed, AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from datasets import load_dataset, load_from_disk, load_metric
import pandas as pd
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]



> ### 1.2 Data loading and preparation:



In [7]:
from datasets import load_dataset
datasets = load_dataset('samsum')

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [8]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [9]:
print("Dialogue:")
print(datasets["train"][1]["dialogue"])

print("\nSummary:")
print(datasets["train"][1]["summary"])

Dialogue:
Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great

Summary:
Olivia and Olivier are voting for liberals in this election. 


## 2. Training the Pegasus Model:



> ### 2.1 Preprocessing and tokenization:



In [10]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [11]:
dataset_samsum_pt = datasets.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]



> ### 2.2 Data Collator and Training Arguments:



In [14]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [16]:
trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=50, save_steps=1e6,
    gradient_accumulation_steps=16
)



> ### 2.3 Training and Model Saving:



In [17]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"])

In [18]:
trainer.train()

You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,2.674700,2.324083
100,2.551900,1.951069
150,2.206500,1.770193
200,1.941600,1.677494
250,1.825500,1.615295
300,1.687200,1.573466
350,1.721200,1.544563
400,1.708400,1.519159
450,1.643700,1.502566
500,1.659900,1.483296


TrainOutput(global_step=920, training_loss=1.8251974468645842, metrics={'train_runtime': 3473.8481, 'train_samples_per_second': 4.241, 'train_steps_per_second': 0.265, 'total_flos': 5528248038285312.0, 'train_loss': 1.8251974468645842, 'epoch': 1.0})

In [19]:
path = "/content/drive/My Drive/Text_Summarization/pegasus-samsum-model"
model_pegasus.save_pretrained(path)

In [20]:
path = "/content/drive/My Drive/Text_Summarization/tokenizer"
tokenizer.save_pretrained(path)

('/content/drive/My Drive/Text_Summarization/tokenizer/tokenizer_config.json',
 '/content/drive/My Drive/Text_Summarization/tokenizer/special_tokens_map.json',
 '/content/drive/My Drive/Text_Summarization/tokenizer/spiece.model',
 '/content/drive/My Drive/Text_Summarization/tokenizer/added_tokens.json',
 '/content/drive/My Drive/Text_Summarization/tokenizer/tokenizer.json')

## 3. Evaluation:



> ### 3.1 Metric calculation and batching:



In [21]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i: i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, batch_size=16, device=device, column_text = "article",
                                column_summary="highlights"):
  article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
  target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

  for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
      inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
      summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                 attention_mask=inputs["attention_mask"].to(device),
                                 length_penalty=0.8, num_beams=8, max_length=128)

      decoded_summaries = [tokenizer.decode(s, skip_special_tokens =True,
                                            clean_up_tokenization_spaces=True)
                            for s in summaries]

      decoded_summaries = [d.replace(""," ") for d in decoded_summaries]
      metric.add_batch(predictions=decoded_summaries, references=target_batch)

  score = metric.compute()
  return score

In [22]:
rouge_names = ["rouge1","rouge2","rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

<ipython-input-22-b23a0b770e04>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')




> ### 3.2 Rouge Score on Test Data:



In [23]:
score = calculate_metric_on_test_ds(
    datasets['test'], rouge_metric, trainer.model, tokenizer, batch_size = 2,
    column_text = 'dialogue', column_summary = 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index = [f'pegasus'])

100%|██████████| 410/410 [13:55<00:00,  2.04s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.018762,0.000342,0.018649,0.018696


## 4. Loading the Trained Model:



> ### 4.1 Tokenizer and Model Loading:



In [24]:
path = "/content/drive/My Drive/Text_Summarization/tokenizer"
tokenizer = AutoTokenizer.from_pretrained(path)

In [25]:
device = "cuda" if torch.cuda.is_available() else "cpu"
path = "/content/drive/My Drive/Text_Summarization/pegasus-samsum-model"
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(path).to(device)

## 5. Prediction:



> ### 5.1 Input dialogue:



In [35]:
gen_kwargs = {"length_penalty": 0.8, "num_beams": 8, "max_length": 60}
dialogue = datasets["test"][12]["dialogue"]
reference_summary = datasets["test"][12]["summary"]
summarize = pipeline("summarization", model=model_pegasus, tokenizer = tokenizer, device=device)

print("Dialogue:")
print(dialogue)

Dialogue:
Charlee: I'm in class. Theatre in Portuguese lol
Curtis: Realllly?
Charlee: Yes. One of my subjects at the university that I attend is portuguese theatre. We are preparing a performance
Curtis: What performance is this? Are you devising it?
Charlee: A polish one translated into portuguese
Curtis: Thats quite cool. Who is the writer?
Charlee: Mrożek




> ### 5.3 Reference summary:



In [36]:
print("Reference Summary:")
print(reference_summary)

Reference Summary:
Charlee is attending Portuguese theater as a subject at university. He and other students are preparing a play by Mrożek translated into Portuguese.




> ### 5.2 Model-generated summary:



In [37]:
print("Model Summary:")
print(summarize(dialogue, **gen_kwargs)[0]["summary_text"])

Model Summary:
Charlee is in Portuguese theatre class. They are preparing a performance. The writer is Mroek, a polish one translated into portuguese.
